In [681]:
import sqlite3
import pandas as pd

In [682]:
# define connection and cursor
conn = sqlite3.connect('store_transactions.db')
c = conn.cursor()

In [683]:
#print tables
pd.read_sql_query("SELECT * FROM stores", conn)

,store_id,location,manager
0,21,"Sydney, NSW",Charles
1,64,"Hobbart, TAS",David
2,79,"Perth, WA",Ellie
3,95,"Melbourne, VIC",Steve


In [684]:
pd.read_sql_query("SELECT * FROM purchases", conn)

,purchase_id,store_id,units_ordered,unit_price
0,4,21,55,16.40
1,5,21,63,14.70
2,23,64,2,21.20
3,47,95,10,11.50
4,54,21,6,3.67
5,65,95,16,19.30


In [685]:
#Sort and Filter
filter = """
SELECT purchase_id, unit_price, units_ordered 
FROM purchases
--WHERE store_id = 21
WHERE store_id IN (21,64) AND unit_price > 10
ORDER BY unit_price DESC;
"""
pd.read_sql_query(filter, conn)

,purchase_id,unit_price,units_ordered
0,23,21.2,2
1,4,16.4,55
2,5,14.7,63


In [686]:
#wildcard for prefix/suffix/fill. eg. %pizza, pizza%, %pizza%, S%E (slow)
filter = """
SELECT * FROM stores
WHERE manager LIKE 'S%';
"""
pd.read_sql_query(filter, conn)


,store_id,location,manager
0,95,"Melbourne, VIC",Steve


In [687]:
#Maths operations
filter = """
SELECT 
    purchase_id, 
    unit_price, 
    units_ordered, 
    unit_price*units_ordered AS total_revenue
FROM purchases
WHERE total_revenue > 300
"""
pd.read_sql_query(filter, conn)

,purchase_id,unit_price,units_ordered,total_revenue
0,4,16.4,55,902.0
1,5,14.7,63,926.1
2,65,19.3,16,308.8


In [688]:
#aggregate functions
filter = """
SELECT 
    COUNT(DISTINCT purchase_id) AS total_transactions, 
    AVG(unit_price) AS avg_price, 
    SUM(units_ordered) AS total_orders
FROM purchases
WHERE store_id=21
"""
pd.read_sql_query(filter, conn)

,total_transactions,avg_price,total_orders
0,3,11.59,124


In [689]:
#Grouping and filter
filter = """
SELECT 
    store_id, 
    COUNT(DISTINCT purchase_id) AS total_transactions
FROM purchases
WHERE store_id>50
GROUP BY store_id
HAVING total_transactions > 1; --WHERE works with rows; HAVING works with groups
"""
pd.read_sql_query(filter, conn)

,store_id,total_transactions
0,95,2


In [690]:
#Subquerries: helps merge data from multiple tables 
#show location of stores where units_ordered > 10
filter = """
SELECT 
    store_id, 
    location, 
    (SELECT SUM(units_ordered) -- only one column can be return in a subquerry
     FROM purchases AS p
     WHERE units_ordered > 10
     GROUP BY store_id 
     HAVING s.store_id = p.store_id) AS total_orders 
FROM stores AS s
WHERE total_orders IS NOT NULL;
"""
pd.read_sql_query(filter, conn)

,store_id,location,total_orders
0,21,"Sydney, NSW",118
1,95,"Melbourne, VIC",16


In [691]:
#joins
join = """
SELECT 
    s.store_id, 
    s.location, 
    SUM(p.units_ordered) AS total_orders
FROM stores s INNER JOIN purchases p -- INNER, LEFT, RIGHT, OUTER
ON s.store_id = p.store_id
WHERE p.units_ordered > 10
GROUP BY s.store_id 
"""
pd.read_sql_query(join, conn)

,store_id,location,total_orders
0,21,"Sydney, NSW",118
1,95,"Melbourne, VIC",16


In [692]:
#union
union = """
SELECT store_id FROM stores
UNION
SELECT store_id FROM purchases
"""
pd.read_sql_query(union, conn)

,store_id
0,21
1,64
2,79
3,95


In [693]:
#Temporary tables
create_temptable = """
CREATE TEMPORARY TABLE store_revenue AS
    SELECT 
        store_id, 
        SUM(unit_price*units_ordered) AS total_revenue
    FROM purchases
    GROUP BY store_id;
"""

calc_revenue = """
SELECT 
    s.manager,
    r.total_revenue
FROM stores s INNER JOIN store_revenue r
ON s.store_id = r.store_id

"""
c.execute(create_temptable)
pd.read_sql_query(calc_revenue, conn)

,manager,total_revenue
0,Charles,1850.12
1,David,42.40
2,Steve,423.80


In [694]:
#Text strings
filter = """
SELECT 
    store_id,
    manager || TRIM('      @     ') || UPPER(SUBSTR(location,1,3)) AS info 
FROM stores
"""
pd.read_sql_query(filter, conn)

,store_id,info
0,21,Charles@SYD
1,64,David@HOB
2,79,Ellie@PER
3,95,Steve@MEL


In [695]:
##Date-time
filter = """
SELECT  DATE('now') AS Date, 
        TIME('now') AS Time, 
        DATETIME('now') AS Date_Time, 
        STRFTIME('%Y/%m/%d', 'now') AS Date_Formatted,
        DATETIME('now') - DATETIME('2021-01-01') AS Date_Diff;
"""
pd.read_sql_query(filter, conn)

,Date,Time,Date_Time,Date_Formatted,Date_Diff
0,2024-07-01,11:48:21,2024-07-01 11:48:21,2024/07/01,3


In [696]:
#CASE (like if-else)
filter = """
SELECT 
    purchase_id,
    units_ordered,
    CASE
        WHEN units_ordered > 20 THEN 'OK'
        ELSE 'Not OK' 
    END AS Status
FROM purchases
"""
pd.read_sql_query(filter, conn)

,purchase_id,units_ordered,Status
0,4,55,OK
1,5,63,OK
2,23,2,Not OK
3,47,10,Not OK
4,54,6,Not OK
5,65,16,Not OK


In [697]:
#Views -- A pre-defined query that generates data only when accessed. 
#It is immutable and does not create copies/store data (unlike temporary tables). 

create_view = """
CREATE VIEW store_revenue AS
    SELECT 
        store_id, 
        SUM(unit_price*units_ordered) AS total_revenue
    FROM purchases
    GROUP BY store_id;
"""

calc_revenue = """
SELECT 
    s.manager,
    r.total_revenue
FROM stores s INNER JOIN store_revenue r
ON s.store_id = r.store_id

"""
c.execute(create_view)
pd.read_sql_query(calc_revenue, conn)

,manager,total_revenue
0,Charles,1850.12
1,David,42.40
2,Steve,423.80


In [698]:
conn.commit()
conn.close()